In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- CONFIGURACIÓN VISUAL DEL SPRINT 3 ---
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.family'] = 'sans-serif'

# --- 1. MOTOR DE BÚSQUEDA DE ARCHIVOS (Solución Path) ---
def buscar_archivo_recursivo(nombre_parcial):
    """
    Busca un archivo en TODA la carpeta del proyecto y subcarpetas
    que contenga el nombre indicado (ej. 'clientes').
    """
    directorio_base = os.getcwd()
    
    for root, dirs, files in os.walk(directorio_base):
        for file in files:
            # Buscamos coincidencias (.csv) ignorando mayúsculas/minúsculas
            if nombre_parcial.lower() in file.lower() and file.endswith('.csv'):
                return os.path.join(root, file)
    return None

# --- 2. CARGA Y FUSIÓN DE DATOS (ETL) ---
def cargar_datos_sprint_3():
    print("🚀 SPRINT 3: Iniciando carga inteligente de datos...")
    
    claves = ['clientes', 'detalle_ventas', 'productos', 'ventas']
    dfs = {}
    
    for k in claves:
        ruta = buscar_archivo_recursivo(k)
        if ruta:
            print(f"   ✅ Archivo '{k}' encontrado en: .../{os.path.basename(ruta)}")
            # Intentamos leer con diferentes codificaciones para evitar errores
            try:
                dfs[k] = pd.read_csv(ruta, encoding='utf-8')
            except UnicodeDecodeError:
                dfs[k] = pd.read_csv(ruta, encoding='latin-1')
        else:
            print(f"   ❌ ERROR CRÍTICO: No se encontró el archivo para '{k}'.")
            return None

    # Asignamos nombres cortos
    df_cli = dfs['clientes']
    df_det = dfs['detalle_ventas']
    df_prod = dfs['productos']
    df_ven = dfs['ventas']

    # Conversión de fechas
    df_ven['fecha'] = pd.to_datetime(df_ven['fecha'])

    print("🔄 Unificando tablas (Data Merging)...")
    # Fusión 1: Detalle con Ventas (Inner Join)
    df = pd.merge(df_det, df_ven, on='id_venta', how='inner')
    # Fusión 2: Agregar Productos (Left Join) - Suffix para evitar duplicados
    df = pd.merge(df, df_prod, on='id_producto', how='left', suffixes=('', '_prod_dup'))
    # Fusión 3: Agregar Clientes (Left Join)
    df = pd.merge(df, df_cli, on='id_cliente', how='left', suffixes=('', '_cli_dup'))
    
    print(f"✅ Dataset Maestro Generado: {df.shape[0]} filas listas para análisis.")
    return df

# --- 3. ANÁLISIS ESPECÍFICOS DEL SPRINT 3 ---

def analisis_categorias(df):
    """Muestra qué categoría de productos genera más ingresos."""
    print("\n📊 Generando Análisis de Categorías...")
    ventas_cat = df.groupby('categoria')['importe'].sum().sort_values(ascending=False)
    
    plt.figure()
    colores = sns.color_palette("viridis", len(ventas_cat))
    bars = plt.bar(ventas_cat.index, ventas_cat.values, color=colores)
    
    plt.title('Ingresos Totales por Categoría', fontsize=14)
    plt.ylabel('Ventas ($)')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Etiquetas de valor
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'${int(height):,}', ha='center', va='bottom', fontsize=9)
    plt.show()

def analisis_medios_pago(df):
    """Muestra la preferencia de pago (contando facturas únicas)."""
    print("\n💳 Generando Análisis de Medios de Pago...")
    # Importante: Eliminar duplicados de id_venta para contar transacciones, no productos
    pagos = df.drop_duplicates(subset='id_venta')['medio_pago'].value_counts()
    
    plt.figure(figsize=(7, 7))
    plt.pie(pagos, labels=pagos.index, autopct='%1.1f%%', startangle=140, 
            colors=sns.color_palette("pastel"), wedgeprops={'edgecolor': 'white'})
    plt.title('Distribución de Medios de Pago (Transacciones)', fontsize=14)
    plt.show()

def analisis_geografico(df):
    """Muestra el Top de ciudades por volumen de venta."""
    print("\n📍 Generando Análisis Geográfico...")
    ventas_ciudad = df.groupby('ciudad')['importe'].sum().sort_values(ascending=False)
    
    plt.figure()
    sns.barplot(x=ventas_ciudad.values, y=ventas_ciudad.index, palette="coolwarm")
    plt.title('Ranking de Ventas por Ciudad', fontsize=14)
    plt.xlabel('Facturación Total ($)')
    plt.show()

# --- EJECUCIÓN PRINCIPAL ---
if __name__ == "__main__":
    df_maestro = cargar_datos_sprint_3()
    
    if df_maestro is not None:
        analisis_categorias(df_maestro)
        analisis_medios_pago(df_maestro)
        analisis_geografico(df_maestro)

📂 Buscando archivos en: 'c:\Users\Miguel\Desktop\Miguelon\GH\Formacion_IA_DataScience_ML\Sprint-3\Data'...

❌ ERROR DE CARGA: Falta el archivo de 'clientes'.
💡 SOLUCIÓN: Verifica que la variable CARPETA_DATOS al inicio del código sea correcta.
